In [2]:
import pandas as pd
import numpy as np

In [3]:
df8 = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - №4 ШЖОББМ.xlsx", sheet_name='data')

In [4]:
for index, subject in enumerate(df8['subject']):
    if not '4' in subject:
        df8.loc[index, 'subject'] = '8' + df8.loc[index, 'subject']

In [5]:
file_path = r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - №4 ШЖОББМ.xlsx"
# df8.to_excel(file_path, index=False, sheet_name='data')

---

In [73]:
def fio_change(df):
    df['fio'] = df['fio'].apply(lambda x: ' '.join(str(x).split()[:2]) if len(str(x).split()) > 2 else x)
    return df

In [74]:
db = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\63 сш.xlsx").map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)
db = fio_change(db)

In [100]:
file_paths = {
    'df1' : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Аққулы СОР и СОЧ.xlsx",
    'df2' : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Баянаул, З.Акишев СОР и СОЧ.xlsx",
    'df3' : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Тереңкөл СОР, СОЧ-1.xlsx"
}

subject_map = {
    'английский язык' : 'ENFL',
    'ағылшын тілі' : 'ENFL',
    'биология' : 'BIOL',
    'география' : 'GEOG',
    'жаратылыстану' : 'SCIE',
    'казахский язык' : 'KZFL',
    'математика' : 'MATH',
    'русский язык' : 'RUFL',
    'русский язык и литература' : 'RUFL',
    'физика' : 'PHYS',
    'химия' : 'CHEM',
    'қазақ тілі' : 'KZFL'
}

dfs = {}
for key, path in file_paths.items():
    dfs[key] = pd.read_excel(path, sheet_name='data')
    dfs[key] = dfs[key].map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)
    dfs[key]['subject'] = dfs[key]['subject'].str.lower()
    dfs[key]['subject'] = dfs[key]['subject'].map(subject_map)
    dfs[key] = fio_change(dfs[key])

In [111]:
merged_dict = {}
for key, df in dfs.items():
    merged = pd.merge(df, db, on='fio', how='left')
    # t = merged[['fio', 'lang']]
    # print(t[t.isna().any(axis=1)]['fio'].unique())

    for index, lang in enumerate(merged['lang']):
        if (lang == 1) and (merged.loc[index, 'subject_x'] == 'RUFL'):
            merged.loc[index, 'subject_x'] = 'RUSL'
        elif (lang == 2) and (merged.loc[index, 'subject_x'] == 'KZFL'):
            merged.loc[index, 'subject_x'] = 'KZSL'
    
    # for fio, lang in zip(merged['fio'], merged['lang']):
    #     if lang == 1:
    #         df.loc[(df['fio'] == fio) & (df['subject_x'] == 'RUFL'), 'subject_x'] == 'RUSL'
    #     elif lang == 2:
    #         df.loc[(df['fio'] == fio) & (df['subject_x'] == 'KZFL'), 'subject_x'] == 'KZSL'
    merged_dict[key] = merged

In [112]:
merged_dict['df2'][merged_dict['df2']['subject_x'] == 'RUSL']

,fio,subject_x,grade_x,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,...,school_ru,grade_y,liter,subject_y,lang,lang_test,iin,dich,score,note
2,Аужан Ерназар,RUSL,8,7,10,12.0,17.0,NaN,NaN,19,...,"КГУ ""СОШ им. З. Акишева""",8.0,8А,08BIOL,1.0,1.0,9.100155e+10,01100000001000000000,3.0,63.0
3,Аужан Ерназар,RUSL,8,7,10,12.0,17.0,NaN,NaN,19,...,"КГУ ""СОШ им. З. Акишева""",8.0,8А,08PHYS,1.0,1.0,9.100155e+10,01011011010101000011,10.0,63.0
18,Бейсетай Мансұр,RUSL,8,6,10,11.0,17.0,NaN,NaN,19,...,"КГУ ""СОШ им. З. Акишева""",8.0,8А,08MATH,1.0,1.0,1.008226e+11,10011000001000010011,7.0,63.0
19,Бейсетай Мансұр,RUSL,8,6,10,11.0,17.0,NaN,NaN,19,...,"КГУ ""СОШ им. З. Акишева""",8.0,8А,08RUSL,1.0,2.0,1.008226e+11,11010100101100000109,8.0,63.0
20,Бейсетай Мансұр,RUSL,8,6,10,11.0,17.0,NaN,NaN,19,...,"КГУ ""СОШ им. З. Акишева""",8.0,8А,08ENFL,1.0,3.0,1.008226e+11,00010011011100001000,7.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,Ынтықбай Дархан,RUSL,8,11,13,12.0,14.0,NaN,NaN,28,...,"КГУ ""СОШ им. З. Акишева""",8.0,8Ә,08RUSL,1.0,2.0,1.005266e+11,11010110010000110011,10.0,63.0
1021,Ынтықбай Дархан,RUSL,8,11,13,12.0,14.0,NaN,NaN,28,...,"КГУ ""СОШ им. З. Акишева""",8.0,8Ә,08ENFL,1.0,3.0,1.005266e+11,11000100001001001010,7.0,63.0
1022,Ынтықбай Дархан,RUSL,8,11,13,12.0,14.0,NaN,NaN,28,...,"КГУ ""СОШ им. З. Акишева""",8.0,8Ә,08MATH,1.0,1.0,1.005266e+11,10011100110101000100,9.0,63.0
1023,Ынтықбай Дархан,RUSL,8,11,13,12.0,14.0,NaN,NaN,28,...,"КГУ ""СОШ им. З. Акишева""",8.0,8Ә,08CHEM,1.0,1.0,1.005266e+11,11100101100011111011,13.0,63.0


In [70]:
dfs['df1'].columns

Index(['fio', 'subject', 'grade', 'sor1', 'sor1_max', 'sor2', 'sor2_max',
       'sor3', 'sor3_max', 'soch', 'soch_max'],
      dtype='object')

In [36]:
subject_set = set()
for key, df in dfs.items():
    subjects = df['subject'].unique()
    for subject in subjects:
        subject_set.add(subject)

In [ ]:
df8

In [24]:
dfs

{'df1':                  fio       subject  sor1  sor1_max  sor2  sor2_max  sor3  \
 0     Бақытбек Бинур      биология    15        25  13.0      14.0   9.0   
 1     Бақытбек Бинур    математика    12        13   NaN       NaN   NaN   
 2     Бақытбек Бинур        физика    12        13  12.0      13.0   NaN   
 3     Бақытбек Бинур     география     9        10   8.0       9.0  10.0   
 4     Бақытбек Бинур  русский язык     9         9  13.0      13.0   NaN   
 ..               ...           ...   ...       ...   ...       ...   ...   
 200  Юлдашев Бахтжан      биология    15        19   7.0      11.0  15.0   
 201  Юлдашев Бахтжан    математика    10        15   NaN       NaN   NaN   
 202  Юлдашев Бахтжан        физика     8        12   9.0      13.0   NaN   
 203  Юлдашев Бахтжан     география     4         8   5.0       9.0   8.0   
 204  Юлдашев Бахтжан  русский язык     8        10   8.0       9.0   NaN   
 
      sor3_max  soch  soch_max  
 0        14.0    23        30  
 